<a href="https://colab.research.google.com/github/GP-FCI-SU/Breast-Cancer-Detection-with-Convolutional-Neural-Networks/blob/master/model_5_d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
local_zip = '/content/drive/My Drive/ddsm-mammography.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/ddsm')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
tf.__version__

'2.2.0'

In [ ]:
features={#needed features to feed into our model  
            
            'label_normal': tf.io.FixedLenFeature([], tf.int64),
            'image': tf.io.FixedLenFeature([], tf.string)
        }#we have also the label feature but not used here

#make a function that will applied to the map transformation
def _parse_image_function(example_proto):
  parsed=tf.io.parse_single_example(example_proto,features)
  image = tf.io.decode_raw(parsed["image"], tf.uint8)
  image = tf.cast(image, tf.float32)/255.0
  image = tf.reshape(image, shape=[299,299,1])#our image shape and model expected one
  #image = tf.divide(iamge, 255.0)
  image = tf.repeat(image, repeats = 3, axis = -1)
  label = tf.cast(parsed["label_normal"], tf.int32) 
  return image,label #atuple of two tensos

In [ ]:
def get_dataset(filenames):
  dataset = tf.data.Dataset.list_files(filenames)
  dataset = dataset.interleave(tf.data.TFRecordDataset,  num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  #dataset = dataset.cache() # This dataset fits in RAM
  #dataset = dataset.repeat()
  #dataset = dataset.shuffle(2048)
  dataset = dataset.batch(batch_size=32) # drop_remainder will be needed on TPU
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
def format_example(image, label):
  image= tf.cast(image, tf.float32)/255.0
  image = tf.repeat(image, repeats = 3, axis = -1)
  return image, label
def load_val():
   X_cv = np.load( "/tmp/ddsm/test10_data/test10_data.npy")
   labels = tf.cast(np.load( "/tmp/ddsm/test10_labels.npy"),tf.int8)
   y_cv = np.zeros(len(labels))
   y_cv[labels != 0] = 1
   # X_cv = (X_cv - 127.0) / 255.0
  # X_cv = X_cv / 255.0
  # X_cv= tf.cast(X_cv, tf.float32)/255.0
  # X_cv = np.repeat(X_cv, repeats = 3, axis = -1)
   return X_cv, y_cv
def get_test_dataset():
  test_dataset = tf.data.Dataset.from_tensor_slices(load_val())
  test_dataset=test_dataset.map(format_example)
  test_dataset= test_dataset.batch(batch_size=32)
  
  return test_dataset

test_dataset=get_test_dataset()
def get_validation_dataset():
  return test_dataset

#get_validation_dataset()


In [ ]:
def get_training_dataset():
  return get_dataset(["/tmp/ddsm/training10_0/training10_0.tfrecords",
                     "/tmp/ddsm/training10_1/training10_1.tfrecords",
                     "/tmp/ddsm/training10_2/training10_2.tfrecords",
                     "/tmp/ddsm/training10_3/training10_3.tfrecords",
                     "/tmp/ddsm/training10_4/training10_4.tfrecords"]
                     )
#def get_test_dataset():
 # test_dataset = tf.data.Dataset.from_tensor_slices(load_val())
  #test_dataset= test_dataset.batch(batch_size=16)
  #test_dataset=test_dataset.cache()
  #return test_dataset

#test_dataset=get_test_dataset()
#def get_validation_dataset():
 # return test_dataset

#get_training_dataset()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if epoch>13:
        self.model.save("/tmp/model_5_d_e{},{:.2f},{:.2f},{:.2f},{:.2f}.h5".format(epoch,logs['accuracy'],logs['loss'],logs['val_accuracy'],logs['val_loss']))

In [ ]:
class_weight = {1: 0.60,
                 0: 0.40}

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=(299,299,3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = True

fine_tune_at = 100 #t0 155

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False


In [ ]:
for image_batch, label_batch in get_training_dataset().take(1):
   pass
image_batch.shape
feature_batch = base_model(image_batch)
print(feature_batch.shape)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

FC1 = tf.keras.layers.Dense(1024,activation='relu')
FC1_batch = FC1(feature_batch_average)
print(FC1_batch.shape)
FC2 = tf.keras.layers.Dense(2048,activation='relu')
FC2_batch = FC2(FC1_batch)
print(FC2_batch.shape)
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(FC2_batch)
print(prediction_batch.shape)



(32, 10, 10, 1280)
(32, 1280)
(32, 1024)
(32, 2048)
(32, 1)


In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  FC1,
  FC2,
  prediction_layer
])



In [ ]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
new_model=tf.keras.models.load_model('/content/drive/My Drive/models/model_5_d_e=20.h5')

In [ ]:
#callback=myCallback()

history = model.fit(
    get_training_dataset(),
    validation_data=get_validation_dataset(),
      epochs=5,
      verbose=1,
 #      callbacks=[callback],
      class_weight=class_weight,
      use_multiprocessing=True
      )

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']


In [ ]:
new_model.save(filepath='/content/drive/My Drive/models/model_5_d_e=67.h5')

In [ ]:
plt.plot(acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
yhat=model.predict_classes(get_validation_dataset() )

yhat = yhat[:, 0]
#_,y=load_val()
#y=tf.cast(y,tf.int8)
#recall = recall_score(y_true, yhat)
#print('Recall: %f' % recall)


In [ ]:
#recall = recall_score(y_true, yhat)
#print('Recall: %f' % recall)
print(classification_report(y, yhat))


In [ ]:
i=60

with open('/content/drive/My Drive/models/res_5_c.txt', 'a+') as filehandle:
    for a,l,va,vl in zip(acc,loss,val_acc,val_loss):
        filehandle.write('{},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(i,a,l,va,vl))
        i+=1

In [ ]:

for y_tr,yh in zip(y,yhat):
  print(str(int(y_tr))+'---'+str(yh))
  
  

Streaming output truncated to the last 5000 lines.
0---0
0---0
0---0
0---0
1---1
0---0
0---0
0---0
0---0
0---0
0---0
0---0
1---0
0---0
0---0
0---0
0---0
0---0
1---1
0---0
0---0
1---1
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
1---1
0---0
1---1
0---0
0---0
0---0
0---0
0---0
1---1
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
1---1
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
1---0
1---1
0---0
0---0
0---0
1---0
0---0
1---1
1---1
0---0
0---0
0---0
1---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
1---0
0---0
0---0
0---0
0---0
0---0
1---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
1---1
1---1
0---0
1---1
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
1---1
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
0---0
1---1
0---0
0---0
0---0
0---0
1---1
0---0
0---0
0---0
0---0
1---0
0---0
0---0
0---0
0

In [ ]:
print(len(y))
print(count)

7682
1002


In [ ]:
acc=acc[:25]
val_acc=val_acc[:25]
loss=loss[:25]
val_loss=val_loss[:25]
